# メモ
- 打者&投手情報を考慮

# 特徴量
## pitch
- 試合内投球数
- イニング
- イニング内打席数
- 打席内投球数
- 投手投球左右
- 投手役割
- 打者打席左右
- 打者打順
- プレイ前アウト数
- プレイ前ボール数
- プレイ前ストライク数
- プレイ前走者状況

## player
### 打者&投手
- 育成選手F
- 位置
- 身長
- 体重
- 年俸
- 出身国

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

df_tr_pitch = pd.read_csv('../data/train_pitch.csv')
df_tr_player = pd.read_csv('../data/train_player.csv')
df_te_pitch = pd.read_csv('../data/test_pitch.csv')
df_te_player = pd.read_csv('../data/test_player.csv')
df_smp = pd.read_csv('../data/sample_submit_ball_type.csv', header=None)

In [24]:
import sys
sys.path.append("..")

import tqdm
import time
import preprocess
import importlib
importlib.reload(preprocess)
cnvrt = preprocess.Converter()
df_tr = cnvrt.convert_df(df_tr_pitch, df_tr_player)

In [25]:
for x in df_tr.keys():
    if x not in ["", "_pitcher", "_batter"]:
        print(x)

num_data
ball_type
corse
num_game_throw
ord_inning
num_inning_bat
num_pitch_in_bat
pitcher_id
pitcher_lr
pitcher_role
batter_id
bat_lr
ord_bat
num_out
num_ball
num_strike
runner_state
cat_pitcher_lr
cat_pitcher_role
cat_bat_lr
cat_runner_state
player_id_batter
training_player_batter
position_batter
num_height_batter
num_weight_batter
num_annual_salary_batter
country_batter
cat_training_player_batter
cat_position_batter
cat_country_batter
player_id_pitcher
training_player_pitcher
position_pitcher
num_height_pitcher
num_weight_pitcher
num_annual_salary_pitcher
country_pitcher
cat_training_player_pitcher
cat_position_pitcher
cat_country_pitcher


In [26]:
features = [
    "num_game_throw",
    "ord_inning",
    "num_inning_bat",
    "num_pitch_in_bat",
    "cat_pitcher_lr",
    "cat_pitcher_role",
    "cat_bat_lr",
    "ord_bat",
    "num_out",
    "num_ball",
    "num_strike",
    "cat_pitcher_lr",
    "cat_training_player_batter",
    "cat_position_batter",
    "num_height_batter",
    "num_weight_batter",
    "num_annual_salary_batter",
    "num_height_pitcher",
    "num_weight_pitcher",
    "num_annual_salary_pitcher",
    "cat_training_player_pitcher",
    "cat_country_pitcher",
    "cat_position_pitcher",
    "cat_country_pitcher",
]

In [27]:
tr_X = df_tr[features]
tr_Y = df_tr["ball_type"]

In [43]:
len(df_te_pitch)

521650

In [28]:
from sklearn.model_selection import KFold

cv_num = 5
kf = KFold(n_splits=cv_num, shuffle=True)

kf_index = []
for tr_i, te_i in kf.split(tr_X):
    kf_index.append([tr_i, te_i])

In [29]:
from sklearn.metrics import log_loss # モデル評価用(logloss) 
import lightgbm as lgb #LightGBM

# params
params = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 8,
    "metric": "multi_logloss",
    "verbose": 2
}

# train
loss_list = []
models = []
for i, kf in enumerate(kf_index):
    train_x = tr_X.iloc[kf[0]]
    train_y = tr_Y.iloc[kf[0]]
    test_x = tr_X.iloc[kf[1]]
    test_y = tr_Y.iloc[kf[1]]
    
    
    train_data = lgb.Dataset(train_x, label=train_y)
    eval_data = lgb.Dataset(test_x, label=test_y)
    
    print(f"train {i}")
    gbm = lgb.train(
        params,
        train_data, 
        valid_sets=eval_data,
        num_boost_round=100,
        verbose_eval=100
    )
    
    models.append(gbm)

train 0
[100]	valid_0's multi_logloss: 1.2282
train 1
[100]	valid_0's multi_logloss: 1.22569
train 2
[100]	valid_0's multi_logloss: 1.22776
train 3
[100]	valid_0's multi_logloss: 1.22442
train 4
[100]	valid_0's multi_logloss: 1.23086


# 予測

In [74]:
df_te = cnvrt.convert_df(df_te_pitch, df_te_player, isTrain=False)
te_X = df_te[features]

In [75]:
result = []
for i, model in enumerate(models):
    start = time.time()
    print(f"start model{i}")
    result.append(model.predict(te_X))
    print(f"end time : {time.time()-start}")

start model0
end time : 17.01969885826111
start model1
end time : 15.994406938552856
start model2
end time : 15.238116025924683
start model3
end time : 14.292888879776001
start model4
end time : 13.647664070129395


In [76]:
from scipy.special import softmax
result = np.array(result)
final_result = np.sum(result, axis=0)
final_result = softmax(final_result, axis=1)

In [77]:
df_submit = pd.concat([df_smp[0], pd.DataFrame(final_result)], axis=1).reset_index(drop=True)
df_submit.columns = range(df_submit.shape[1])
df_submit.to_csv("submits/v2.csv", index=False, header=False)